In [1]:
import nbimporter
from RADARDataCapture import FMCWRADARDataCapture
import h5py
import numpy as np
import imageio
import matplotlib.pyplot as plt
from datetime import datetime

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
# Define the path to the test data
test_path = "/Users/danielcopeland/Library/CloudStorage/OneDrive-MassachusettsInstituteofTechnology/Yoga_Study_RADAR_Data/02/02_CRW2L_RR_V1.h5"

# Initialize the FMCWRADARDataCapture object
test_data = FMCWRADARDataCapture(file_path=test_path)

# Load and save the data
dataCubes = test_data.load_and_save()

print(dataCubes.shape)


(4, 1000, 128, 256)


In [3]:
# Process the data with Range-Doppler processing
processed_data = test_data.range_doppler_processing(dataCubes)

In [4]:
print(processed_data.shape)

(4, 1000, 64, 256)


In [5]:
gif_filename = test_path.split("/")[-1].split(".")[0] + "__" + str(datetime.now()) + '.gif'
print(gif_filename)
gif_path = test_data.create_gif(processed_data[0], gif_filename)
print(f"GIF saved to {gif_path}")

02_CRW2L_RR_V1__2024-01-29 13:47:17.820541.gif
GIF saved to /Users/danielcopeland/Library/Mobile Documents/com~apple~CloudDocs/MIT Masters/DRL/LABx/RADARTreePose/data/gifs/02_CRW2L_RR_V1__2024-01-29 13:47:17.820541.gif
